In [4]:
! python p3.py

Generation 0
Pacman died! Score: -484
Average Score: -484.0
Scores:        -484.0
Win Rate:      0/1 (0.00)
Record:        Loss
Pacman died! Score: -492
Average Score: -492.0
Scores:        -492.0
Win Rate:      0/1 (0.00)
Record:        Loss
Pacman died! Score: -491
Average Score: -491.0
Scores:        -491.0
Win Rate:      0/1 (0.00)
Record:        Loss
Pacman died! Score: -411
Average Score: -411.0
Scores:        -411.0
Win Rate:      0/1 (0.00)
Record:        Loss
Pacman died! Score: -487
Average Score: -487.0
Scores:        -487.0
Win Rate:      0/1 (0.00)
Record:        Loss
Pacman died! Score: -480
Average Score: -480.0
Scores:        -480.0
Win Rate:      0/1 (0.00)
Record:        Loss
Pacman died! Score: -489
Average Score: -489.0
Scores:        -489.0
Win Rate:      0/1 (0.00)
Record:        Loss
Pacman died! Score: -531
Average Score: -531.0
Scores:        -531.0
Win Rate:      0/1 (0.00)
Record:        Loss
Pacman died! Score: -480
Average Score: -480.0
Scores:        -480.

Pacman died! Score: -441
Average Score: -441.0
Scores:        -441.0
Win Rate:      0/1 (0.00)
Record:        Loss
Pacman died! Score: -426
Average Score: -426.0
Scores:        -426.0
Win Rate:      0/1 (0.00)
Record:        Loss
Pacman died! Score: -473
Average Score: -473.0
Scores:        -473.0
Win Rate:      0/1 (0.00)
Record:        Loss
Generation 22
Pacman died! Score: -490
Average Score: -490.0
Scores:        -490.0
Win Rate:      0/1 (0.00)
Record:        Loss
Pacman died! Score: -465
Average Score: -465.0
Scores:        -465.0
Win Rate:      0/1 (0.00)
Record:        Loss
Pacman died! Score: -466
Average Score: -466.0
Scores:        -466.0
Win Rate:      0/1 (0.00)
Record:        Loss
Generation 23
Pacman died! Score: -484
Average Score: -484.0
Scores:        -484.0
Win Rate:      0/1 (0.00)
Record:        Loss
Pacman died! Score: -420
Average Score: -420.0
Scores:        -420.0
Win Rate:      0/1 (0.00)
Record:        Loss
Pacman died! Score: -460
Average Score: -460.0
Score

Pacman died! Score: -428
Average Score: -428.0
Scores:        -428.0
Win Rate:      0/1 (0.00)
Record:        Loss
Pacman died! Score: -431
Average Score: -431.0
Scores:        -431.0
Win Rate:      0/1 (0.00)
Record:        Loss
Generation 45
Pacman died! Score: -483
Average Score: -483.0
Scores:        -483.0
Win Rate:      0/1 (0.00)
Record:        Loss
Pacman died! Score: -386
Average Score: -386.0
Scores:        -386.0
Win Rate:      0/1 (0.00)
Record:        Loss
Pacman died! Score: -447
Average Score: -447.0
Scores:        -447.0
Win Rate:      0/1 (0.00)
Record:        Loss
Generation 46
Pacman died! Score: -482
Average Score: -482.0
Scores:        -482.0
Win Rate:      0/1 (0.00)
Record:        Loss
Pacman died! Score: 9
Average Score: 9.0
Scores:        9.0
Win Rate:      0/1 (0.00)
Record:        Loss
Pacman died! Score: -492
Average Score: -492.0
Scores:        -492.0
Win Rate:      0/1 (0.00)
Record:        Loss
Generation 47
Pacman died! Score: -438
Average Score: -438.0


Pacman died! Score: -355
Average Score: -355.0
Scores:        -355.0
Win Rate:      0/1 (0.00)
Record:        Loss
Pacman died! Score: -446
Average Score: -446.0
Scores:        -446.0
Win Rate:      0/1 (0.00)
Record:        Loss
Generation 68
Pacman died! Score: -463
Average Score: -463.0
Scores:        -463.0
Win Rate:      0/1 (0.00)
Record:        Loss
Pacman died! Score: -482
Average Score: -482.0
Scores:        -482.0
Win Rate:      0/1 (0.00)
Record:        Loss
Pacman died! Score: -447
Average Score: -447.0
Scores:        -447.0
Win Rate:      0/1 (0.00)
Record:        Loss
Generation 69
Pacman died! Score: -448
Average Score: -448.0
Scores:        -448.0
Win Rate:      0/1 (0.00)
Record:        Loss
Pacman died! Score: -375
Average Score: -375.0
Scores:        -375.0
Win Rate:      0/1 (0.00)
Record:        Loss
Pacman died! Score: -457
Average Score: -457.0
Scores:        -457.0
Win Rate:      0/1 (0.00)
Record:        Loss
Generation 70
Pacman died! Score: -457
Average Score

Pacman died! Score: -472
Average Score: -472.0
Scores:        -472.0
Win Rate:      0/1 (0.00)
Record:        Loss
Pacman died! Score: -422
Average Score: -422.0
Scores:        -422.0
Win Rate:      0/1 (0.00)
Record:        Loss
Generation 91
Pacman died! Score: -476
Average Score: -476.0
Scores:        -476.0
Win Rate:      0/1 (0.00)
Record:        Loss
Pacman died! Score: -316
Average Score: -316.0
Scores:        -316.0
Win Rate:      0/1 (0.00)
Record:        Loss
Pacman died! Score: -487
Average Score: -487.0
Scores:        -487.0
Win Rate:      0/1 (0.00)
Record:        Loss
Generation 92
Pacman died! Score: -465
Average Score: -465.0
Scores:        -465.0
Win Rate:      0/1 (0.00)
Record:        Loss
Pacman died! Score: -481
Average Score: -481.0
Scores:        -481.0
Win Rate:      0/1 (0.00)
Record:        Loss
Pacman died! Score: -470
Average Score: -470.0
Scores:        -470.0
Win Rate:      0/1 (0.00)
Record:        Loss
Generation 93
Pacman died! Score: -474
Average Score